# Happy Cow Ice Cream: Data-Driven Sales Forecasting Case Study


## Business Context: Happy Cow Ice Cream Case Study

Happy Cow is a small-batch, premium ice cream store based on a university campus in Hong Kong. Founded by Mary Schroeder, the business aimed to offer healthier, unique ice cream flavors that appealed to a diverse crowd: students, staff, and tourists.

### Operational Strategy
- **Location**: HKU campus – benefiting from a regular student and staff base, but dependent on campus foot traffic.
- **Product**: Unconventional, data-inspired flavors like Ginger, Chai Tea, and Mango Lime Coconut.
- **Data Collection**: PoS system used to track sales by hour, date, flavor, and consumer group.

### Strategic Use of Data
- Mary and Prem (store manager) believed in leveraging data for:
  - Stocking decisions
  - Flavor development
  - Staffing optimization
  - Promotions

### Class Discussion Takeaways
- **Hourly data reveals sales dynamics**: Useful for designing time-specific promotions (e.g., 3–5 PM happy hours).
- **Seasonality matters**: Certain flavors trend higher in summer (e.g., Fruity), others in cooler months (e.g., Warming).
- **Segmentation drives insights**: Student vs. Staff vs. Tourist behavior differs—affecting inventory and promotions.

This notebook uses descriptive time series analysis and segmentation to provide business insights and promotional suggestions grounded in data. While limited in historical range (April–August), the dataset is rich in granularity, supporting tactical decision-making for a growing retail food business.



## 🛠️ Data Preprocessing & Cleaning

- Cleaned and parsed dates from multiple formats (`'Sat 1st Apr 2017'` ➝ `datetime`).
- Merged datasets from different consumer groups into a unified structure.
- Used `.melt()` and `.pivot()` for reshaping.
- Grouped and aggregated sales by date, flavor, and consumer segment.
- Validated data completeness and highlighted zero-sales anomalies using IQR-based outlier detection.


## Time Series Example from Daily Life
**Example:** Daily steps walked over a month.

This time series reflects personal fitness behavior, useful for setting goals, tracking consistency, or detecting health issues.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a time series of daily steps walked by a person over a month
dates = pd.date_range(start="2024-04-01", end="2024-04-30")
steps = np.random.randint(4000, 12000, size=len(dates))
daily_steps = pd.DataFrame({"Date": dates, "Steps": steps})

plt.figure(figsize=(10, 5))
plt.plot(daily_steps["Date"], daily_steps["Steps"], marker="o")
plt.title("Daily Steps Over a Month")
plt.xlabel("Date")
plt.ylabel("Steps")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Q1. Sales Performance by Consumer Group
Visual comparison of daily sales by students, staff, and tourists. Insights help optimize pricing, staffing, and inventory.

In [ ]:

# Import and parse Excel
import pandas as pd
import re

daily_sales_path = "/mnt/data/HK1227-XLS-ENG.xlsx"
student = pd.read_excel(daily_sales_path, sheet_name="student daily")
staff = pd.read_excel(daily_sales_path, sheet_name="staff daily")
tourist = pd.read_excel(daily_sales_path, sheet_name="tourism daily")

# Clean date format
def clean_date_suffix(date_str):
    return re.sub(r"(\d+)(st|nd|rd|th)", r"\1", date_str)

for df in [student, tourist]:
    df["Date"] = pd.to_datetime(df["Date"].apply(clean_date_suffix), format="%a %d %b %Y")
staff.rename(columns={"Product": "Date"}, inplace=True)
staff["Date"] = pd.to_datetime(staff["Date"].apply(clean_date_suffix), format="%a %d %b %Y")

# Melt into long format
student_melt = student.melt(id_vars="Date", var_name="Flavor", value_name="Sales")
student_melt["Group"] = "Student"
staff_melt = staff.melt(id_vars="Date", var_name="Flavor", value_name="Sales")
staff_melt["Group"] = "Staff"
tourist_melt = tourist.melt(id_vars="Date", var_name="Flavor", value_name="Sales")
tourist_melt["Group"] = "Tourist"

# Combine and aggregate
df_all = pd.concat([student_melt, staff_melt, tourist_melt])
sales_by_group = df_all.groupby(["Date", "Group"])["Sales"].sum().reset_index()
pivot = sales_by_group.pivot(index="Date", columns="Group", values="Sales")

# Plot
pivot.plot(figsize=(12,6), marker='o')
plt.title("Daily Ice Cream Sales by Consumer Group")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.grid(True)
plt.tight_layout()
plt.show()


## Q2. Flavor Groupings and Seasonal Insights
Group flavors (e.g., Fruity, Classic, Warming) to visualize seasonal patterns. This helps optimize inventory and marketing.

In [ ]:

flavor_groups = {
    "Fruity": ["Mango", "Strawberry", "Banana", "Lime Coconut", "Apricot"],
    "Nutty": ["Hazelnut", "Pistachio"],
    "Warming": ["Ginger", "Chai Tea"],
    "Classic": ["Chocolate", "Vanilla", "Mint Choco", "Pure Coconut", "S. Caramel"]
}

def map_group(flavor):
    for group, items in flavor_groups.items():
        if any(f.lower() in flavor.lower() for f in items):
            return group
    return "Other"

df_all["FlavorGroup"] = df_all["Flavor"].apply(map_group)
flavor_sales = df_all.groupby(["Date", "FlavorGroup"])["Sales"].sum().reset_index()
flavor_pivot = flavor_sales.pivot(index="Date", columns="FlavorGroup", values="Sales")
flavor_pivot.plot(figsize=(12,6), marker='o')
plt.title("Sales Trends by Flavor Group")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.grid(True)
plt.tight_layout()
plt.show()


## Q3. Outlier Detection
Using the IQR method to identify unusually high/low sales days.

In [ ]:

total_daily = df_all.groupby("Date")["Sales"].sum().reset_index()
Q1 = total_daily["Sales"].quantile(0.25)
Q3 = total_daily["Sales"].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
total_daily["Outlier"] = (total_daily["Sales"] < lower) | (total_daily["Sales"] > upper)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(total_daily["Date"], total_daily["Sales"], label="Sales")
plt.scatter(total_daily[total_daily["Outlier"]]["Date"], 
            total_daily[total_daily["Outlier"]]["Sales"], color='red', label="Outliers")
plt.title("Daily Sales with Outliers Highlighted")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Display outliers
total_daily[total_daily["Outlier"]]


## Q4. Purpose of Time Series Analysis
**Descriptive** due to short data period (5 months).

### Limitations:
- No annual cycle captured, limiting seasonal forecasting.
- Missing academic break trends and weather impacts.
- Not enough data for stable ARIMA or LSTM models.
**Use Case:** Explore trends, behavior segments, and prepare for predictive modeling in future with a longer dataset.

## Q5. Susan’s Promotional Recommendations
- Promote fruity flavors in summer, warming ones in cooler months.
- Offer student-exclusive deals during weekday slumps.
- Loyalty programs (e.g., free scoop after 5 visits).
- Enable flavor voting to drive seasonal rotation.
- Collaborate with local Cantonese health/wellness influencers.
- Create tasting booths to convert skeptics.
- Track performance of each promotion using PoS data.

## Q6. Forecasting Techniques and Operational Impact
**Techniques:**
- Time series analysis by day, flavor, and group.
- Flavor category seasonality tracking.
- Outlier detection for special event planning.
- Segment-level sales analysis.

**Impacts:**
- Improved stock control (prevented shortages).
- Data-driven promotions and campaigns.
- Informed product development and expansion.
- Data-informed decision-making culture.

## Bonus: Hourly Sales Analysis
To complement daily sales analysis, hourly data helps reveal peak hours, slumps, and time-specific promotion opportunities.

In [ ]:

import re

# Load hourly sales data
hourly_sales = pd.read_excel("/mnt/data/HK1228-XLS-ENG.xlsx", sheet_name="Raw data")

# Clean and parse timestamps
def robust_clean_hourly_time(x):
    if isinstance(x, str):
        x = x.lower()
        x = re.sub(r"(\d+)(st|nd|rd|th)", r"\1", x)
        try:
            return pd.to_datetime(x, format="%I%p %a %d %b %Y")
        except:
            return pd.NaT
    return pd.NaT

hourly_sales["DateTime"] = hourly_sales["Time"].apply(robust_clean_hourly_time)
hourly_sales["Hour"] = hourly_sales["DateTime"].dt.hour
hourly_sales["Weekday"] = hourly_sales["DateTime"].dt.day_name()

# Plot average hourly sales
import seaborn as sns
import matplotlib.pyplot as plt

avg_hourly = hourly_sales.groupby("Hour")["Sales"].mean().reset_index()

plt.figure(figsize=(10, 5))
sns.lineplot(data=avg_hourly, x="Hour", y="Sales", marker="o")
plt.title("Average Ice Cream Sales by Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Average Sales")
plt.grid(True)
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()


### Insight:
- Sales are near-zero in early morning hours (12 AM – 9 AM).
- Peaks likely occur during afternoon and early evening (12–6 PM).
- Promotions or staff scheduling can be optimized around these trends.


## 📌 Summary & Recommendations

### Key Findings:
- **Students** show more consistent weekday purchases.
- **Tourists** drive weekend peaks and flavor variety demand.
- **Fruity flavors** spike in summer, validating seasonal product rotation.
- **Hourly trends** indicate strong afternoon performance (12–6 PM), supporting time-based offers.
- **Outliers** in sales were mostly due to store closure days (0 sales) or event spikes.

### Strategic Recommendations:
1. **Time-Specific Promotions**
   - Launch 3–5 PM “Happy Hours” targeting students.
   - Run weekend combo deals for tourists.

2. **Seasonal Flavor Planning**
   - Stock up on Fruity flavors April–July.
   - Introduce Warming flavors like Ginger and Chai Tea in colder months (planned expansion).

3. **Operational Improvements**
   - Use hourly data to schedule staff shifts more efficiently.
   - Investigate 0-sales days to identify closure causes and improve uptime.

4. **Future Data Collection**
   - Expand data logging to include weather, events, and store hours.
   - Collect more months of data to enable predictive modeling and seasonal forecasts.
